## CRUD operations
### Create a new db name Todo and a new collection named "CRUD_exercise" and do the following:

### 1: Take the dict created in the TODO 4 in chapter I and save it in the collection "CRUD_exercise".

In [1]:
pip install pymongo

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pymongo

In [3]:
# Dictionnaire pour l'article de LeCun et al.
lecun_paper = {
    "authors": ["Yann LeCun", "Corinna Cortes", "Christopher J.C. Burges"],
    "title": "MNIST handwritten digit database",
    "affiliations": ["Facebook AI Research (FAIR), New York, USA", "Google Research, Mountain View, USA"]
}

# Dictionnaire pour l'article de Goodfellow et al.
goodfellow_paper = {
    "authors": ["Ian Goodfellow", "Yoshua Bengio", "Aaron Courville"],
    "title": "Deep Learning",
    "affiliations": ["Google Research, Mountain View, USA", "Mila - Quebec AI Institute, Université de Montréal, Montreal, Canada"]
}

import json

# Sauvegarder le dictionnaire dans un fichier JSON
with open('articles.json', 'w') as f:
    json.dump({"LeCun_paper": lecun_paper, "Goodfellow_paper": goodfellow_paper}, f)

# Charger le dictionnaire depuis le fichier JSON
with open('articles.json', 'r') as f:
    loaded_data = json.load(f)

client = pymongo.MongoClient('localhost', 27017)
mydb = client["Todo"]
collection = mydb["CRUD_exercise"]
collection.insert_one(loaded_data)

InsertOneResult(ObjectId('662d3ae2b4991738e6063ee9'), acknowledged=True)

### 2: Insert 3 documents with key = x and values = 1, delete one of them. Which one is deleted first ? the most recent or oldest one ? increment the value of x to 4.

In [4]:
import tqdm
for i in tqdm.tqdm(range(3)):
    post = {"x":1}
    collection.insert_one(post)

100%|██████████| 3/3 [00:00<00:00, 78.98it/s]


In [5]:
# Delete one
collection.delete_one({'x': 1})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [6]:
collection.update_many({'x': 1}, {'$inc': {'x': 3}})

UpdateResult({'n': 6, 'nModified': 6, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

### 3: Insert the dict created in the TODO 6 Chapter I in the example collection.

In [7]:
client = pymongo.MongoClient('localhost', 27017)
mydb = client["Todo"]
collection = mydb["CRUD_exercise"]


In [10]:
import lxml.etree
import xmltodict
import json

xml_file = "C:/Users/oumis/Desktop/Nosql/xml_file2.nxml"
root = lxml.etree.parse(xml_file)
print(lxml.etree.tostring(root, encoding="unicode", pretty_print=True))

with open(xml_file, "rb") as file:
    xml_dict = xmltodict.parse(file)
    print(xml_dict)  

json_data = json.dumps(xml_dict, indent=4)
with open("xml_data.json", "w") as json_file:
    json_file.write(json_data)


client = pymongo.MongoClient('localhost', 27017)
db = client["Todo"]
collection = db["CRUD_exercise"]
collection.insert_one(xml_dict)

<note>
  <date>2015-09-01</date>
  <hour>08:30</hour>
  <to>Tove</to>
  <from>Jani</from>
  <body>Don't forget me this weekend!</body>
</note>

{'note': {'date': '2015-09-01', 'hour': '08:30', 'to': 'Tove', 'from': 'Jani', 'body': "Don't forget me this weekend!"}}


InsertOneResult(ObjectId('662d3bbdb4991738e6063eef'), acknowledged=True)

### 4: Get documents where authors key exist in the collection "CRUD_exercise".

In [14]:
import pymongo

# Connect to MongoDB
client = pymongo.MongoClient('localhost', 27017)

# Select database and collection
db = client["Todo"]
collection = db["CRUD_exercise"]

# Query documents where the "authors" key exists
query = {"authors": {"$exists": True}}
result = collection.find(query)

# Print the documents
for doc in result:
    print(doc)


### 5: Change the documents where x = 4 to x = 1.

In [15]:
client = pymongo.MongoClient('localhost', 27017)
db = client["Todo"]
collection = db["CRUD_exercise"]

# Update documents where x = 4 to x = 1
query = {"x": 4}
update = {"$set": {"x": 1}}
result = collection.update_many(query, update)

# Print the number of documents updated
print("Number of documents updated:", result.modified_count)


Number of documents updated: 6


### 6: Find documents where author is not_mike and set author as real_mike.

In [16]:
doc = collection.find_one_and_update({'author': "not_mike"}, {'$set': {'author': "real_mike"}})
print(doc)

None


In [17]:
import datetime
import pymongo

client = pymongo.MongoClient('localhost', 27017)
mydb = client["Todo"]
collection = mydb["CRUD_exercise"]

# new_posts = list of dicts [{},{}]
new_posts = [{"author": "Mike",
              "title":"Python is fun",
               "text": "Another post!",
               "tags": ["bulk", "insert"],
              # date object format (year,month,day,hour,minute)
               "date": datetime.datetime(2009, 11, 12, 11, 14)},
              {"author": "Eliot",
               "title": "MongoDB is fun",
               "text": "and pretty easy too!",
               "date": datetime.datetime(2009, 11, 10, 10, 45)}]

collection.insert_many(new_posts)

InsertManyResult([ObjectId('662d3c18b4991738e6063ef5'), ObjectId('662d3c18b4991738e6063ef6')], acknowledged=True)

In [18]:
doc=collection.update_one({"author": {"$ne": "mike"}},
                      {'$set': {
                          'author': "real_mike"
                          }
                      }, upsert=False)
print(doc)

UpdateResult({'n': 1, 'nModified': 1, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)


### 7: Delete documents where author is real_mike.

In [19]:
collection.delete_one({'author': "real_mike"})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

## Managing DB

### 8: create a collection named "CRUD_exercise_benchmark" with 500k observations, ids increment of 2 (sequence:0,2,4,6,...1M). Give a random np.array with a key named "values" and use the insert_many. Then create an index on the id and benchmark queries before and after indexing. Did the index help ?

In [20]:
import numpy as np
import pymongo
import tqdm


list_of_insertion = []
for i in tqdm.tqdm(range(1, 500001, 2)):
    # Générer un tableau NumPy aléatoire de taille 10
    values_array = np.random.rand(1)
    
    # Créer le document à insérer
    post = {
        "user_id": i,
        "values": values_array.tolist()  # Convertir le tableau NumPy en liste pour l'insertion dans MongoDB
    }
    
    # Ajouter le document à la liste des insertions
    list_of_insertion.append(post)
    
    # Insérer les documents par lots de 15000
    if i % 15000 == 0:
        collection.insert_many(list_of_insertion)
        list_of_insertion = []

# Insérer les documents restants
if list_of_insertion:
    collection.insert_many(list_of_insertion)

100%|██████████| 250000/250000 [00:01<00:00, 132576.80it/s]


In [21]:
collection.find( { "user_id": 430 } ).explain()['executionStats']

{'executionSuccess': True,
 'nReturned': 0,
 'executionTimeMillis': 0,
 'totalKeysExamined': 0,
 'totalDocsExamined': 0,
 'executionStages': {'stage': 'FETCH',
  'nReturned': 0,
  'executionTimeMillisEstimate': 0,
  'works': 1,
  'advanced': 0,
  'needTime': 0,
  'needYield': 0,
  'saveState': 0,
  'restoreState': 0,
  'isEOF': 1,
  'docsExamined': 0,
  'alreadyHasObj': 0,
  'inputStage': {'stage': 'IXSCAN',
   'nReturned': 0,
   'executionTimeMillisEstimate': 0,
   'works': 1,
   'advanced': 0,
   'needTime': 0,
   'needYield': 0,
   'saveState': 0,
   'restoreState': 0,
   'isEOF': 1,
   'keyPattern': {'user_id': 1},
   'indexName': 'user_id_1',
   'isMultiKey': False,
   'multiKeyPaths': {'user_id': []},
   'isUnique': False,
   'isSparse': False,
   'isPartial': False,
   'indexVersion': 2,
   'direction': 'forward',
   'indexBounds': {'user_id': ['[430, 430]']},
   'keysExamined': 0,
   'seeks': 1,
   'dupsTested': 0,
   'dupsDropped': 0}},
 'allPlansExecution': []}

In [22]:
collection.create_index([ ("user_id",1)])

'user_id_1'

In [23]:
collection.find( { "user_id": 430} ).explain()['executionStats']

{'executionSuccess': True,
 'nReturned': 0,
 'executionTimeMillis': 0,
 'totalKeysExamined': 0,
 'totalDocsExamined': 0,
 'executionStages': {'stage': 'FETCH',
  'nReturned': 0,
  'executionTimeMillisEstimate': 0,
  'works': 1,
  'advanced': 0,
  'needTime': 0,
  'needYield': 0,
  'saveState': 0,
  'restoreState': 0,
  'isEOF': 1,
  'docsExamined': 0,
  'alreadyHasObj': 0,
  'inputStage': {'stage': 'IXSCAN',
   'nReturned': 0,
   'executionTimeMillisEstimate': 0,
   'works': 1,
   'advanced': 0,
   'needTime': 0,
   'needYield': 0,
   'saveState': 0,
   'restoreState': 0,
   'isEOF': 1,
   'keyPattern': {'user_id': 1},
   'indexName': 'user_id_1',
   'isMultiKey': False,
   'multiKeyPaths': {'user_id': []},
   'isUnique': False,
   'isSparse': False,
   'isPartial': False,
   'indexVersion': 2,
   'direction': 'forward',
   'indexBounds': {'user_id': ['[430, 430]']},
   'keysExamined': 0,
   'seeks': 1,
   'dupsTested': 0,
   'dupsDropped': 0}},
 'allPlansExecution': []}

#### Oui car il facilite le traitement et l'execution est plus vite

### 9: create a random collection in a random db and put the new collection in the tutorial DB

In [24]:
import pymongo
import random
import string

# Connect to MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Get a list of existing databases
existing_databases = client.list_database_names()

# Choose a random database name
random_db_name = ''.join(random.choices(string.ascii_lowercase, k=10))
while random_db_name in existing_databases:
    random_db_name = ''.join(random.choices(string.ascii_lowercase, k=10))

# Create a random database
random_db = client[random_db_name]

# Choose a random collection name
random_collection_name = ''.join(random.choices(string.ascii_lowercase, k=10))

# Generate some random data for the collection
random_data = [{"key": ''.join(random.choices(string.ascii_lowercase, k=5)), "value": random.randint(1, 100)} for _ in range(10)]

# Insert data into the random collection
random_collection = random_db[random_collection_name]
random_collection.insert_many(random_data)

# Move the random collection to the "tutorial" database
tutorial_db = client["tutorial"]
tutorial_db[random_collection_name].insert_many(random_collection.find())

# Drop the random collection from the random database
random_db.drop_collection(random_collection_name)

# Print confirmation message
print(f"The collection '{random_collection_name}' has been moved to the 'tutorial' database.")


The collection 'qttazgrach' has been moved to the 'tutorial' database.


### 10: What is the difference between an inner join and an outer join ? Is the query seen during course an inner or outer join ? Play with the query to show all the joins.

In [25]:
import pymongo

client = pymongo.MongoClient('localhost', 27017)
mydb = client["tutorial"]
collection = mydb["benchmark"]

pipeline = [{'$lookup': 
                {'from' : 'benchmark_2',
                 'localField' : 'user_id',
                 'foreignField' : 'user_id',
                 'as' : 'cellmodels'}},
            {'$unwind': '$cellmodels'},
            {'$project': 
                {'user_id':1,"user_name":1, 'cellmodels.user_id':1, 'cellmodels.random_value':1}} 
             ]

documents = collection.aggregate(pipeline)
for i in range(20):
    print(next(documents))

{'_id': ObjectId('662a3b95be3f945beb6bc012'), 'user_id': 1, 'user_name': 'William Freeman', 'cellmodels': {'user_id': 1, 'random_value': 100}}
{'_id': ObjectId('662a3b95be3f945beb6bc014'), 'user_id': 3, 'user_name': 'David Fowler', 'cellmodels': {'user_id': 3, 'random_value': 300}}
{'_id': ObjectId('662a3b95be3f945beb6bc016'), 'user_id': 5, 'user_name': 'Maria Baker', 'cellmodels': {'user_id': 5, 'random_value': 500}}
{'_id': ObjectId('662a3b95be3f945beb6bc018'), 'user_id': 7, 'user_name': 'Danny Swafford', 'cellmodels': {'user_id': 7, 'random_value': 700}}
{'_id': ObjectId('662a3b95be3f945beb6bc01a'), 'user_id': 9, 'user_name': 'Theresa Athayde', 'cellmodels': {'user_id': 9, 'random_value': 900}}
{'_id': ObjectId('662a3b95be3f945beb6bc01c'), 'user_id': 11, 'user_name': 'Robert Head', 'cellmodels': {'user_id': 11, 'random_value': 1100}}
{'_id': ObjectId('662a3b95be3f945beb6bc01e'), 'user_id': 13, 'user_name': 'Artie Masterson', 'cellmodels': {'user_id': 13, 'random_value': 1300}}
{'_id

## Real world problems


### 11: Use the oaipmh and api code get papers after January 2020 and for "cs,math,econ" categories. Insert them in MongoDB. Import only the first 200. How is it sorted ? How can you define your own sort()? Query papers to get papers after 2021, which have 3 authors and with domain "cs".

In [56]:
pip install feedparser

  Obtaining dependency information for feedparser from https://files.pythonhosted.org/packages/7c/d4/8c31aad9cc18f451c49f7f9cfb5799dadffc88177f7917bc90a66459b1d7/feedparser-6.0.11-py3-none-any.whl.metadata
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/81.3 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.3 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 81.3/81.3 kB 1.5 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6061 sha256=1ed33ee139448a18d92ae7172673e0781151bc39cf001c2afbc97e30f489e4a0
  Stored in directory: c:\users\oumis\appdata\local\pip\cache\wheels\3b\25\2a\105d6a15df6914f4d15047691c6c28f9052cc1173e40285d03
Successfully built sgmllib3k
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests
import feedparser
import tqdm
import time
import pymongo

# For each id get all the metadata https://info.arxiv.org/help/api/basics.html#python_simple_example

client = pymongo.MongoClient('localhost',27017)
mydb = client["tutorial"]
collection = mydb["arxiv_api"]

# get list of ids previously downloaded
with open("C:/Users/oumis/Desktop/Nosql/arxiv_cs.txt","r") as lines:
    ids = list(set(lines.read().split("\n")[0:-2]))

#init list of ids and iteration
ids_query = []

# loop through ids
for id_ in tqdm.tqdm(ids):
    #append id to list
    ids_query.append(id_)
    # if len list = 100 
    if len(ids_query) == 100 :
        # collapse list of id
        ids_query = ",".join(ids_query)
        # query the api for the 100 ids
        response = requests.get('http://export.arxiv.org/api/query?id_list={}&max_results=100'.format(ids_query))
        # parse response
        feed = feedparser.parse(response.content)
        # commit the 100 papers found
        list_of_insertion = []
        for entry in feed.entries:
            list_of_insertion.append(dict(entry))
        collection.insert_many(list_of_insertion)
        ids_query = []
        time.sleep(1/3)

 12%|█▏        | 72100/617968 [31:16<3:34:23, 42.43it/s] 

### 12: Do the same as exercise 8 but with the connection to the cluster. Then check the metrics and take screenshot of opcounters, logical size and connections.

In [6]:
from pymongo import MongoClient

# Connect to the MongoDB cluster
client = MongoClient('localhost', 27017)

# Access the desired database and collection
db = client["Todo"]
collection = db["CRUD_exercise"]

# Fetch the metrics
opcounters = db.command("serverStatus")["opcounters"]
logical_size = db.command("dbstats")["dataSize"]
connections = db.command("serverStatus")["connections"]

# Print the metrics (you can also save them or take screenshots)
print("Opcounters:", opcounters)
print("Logical Size:", logical_size)
print("Connections:", connections)


Opcounters: {'insert': 11255077, 'query': 449, 'update': 100265, 'delete': 11, 'getmore': 1, 'command': 140991}
Logical Size: 59002157.0
Connections: {'current': 26, 'available': 999974, 'totalCreated': 186, 'rejected': 0, 'active': 8, 'threaded': 26, 'exhaustIsMaster': 0, 'exhaustHello': 6, 'awaitingTopologyChanges': 7}


### 13: Download a random image and store it in a collection.

In [2]:
import requests
from pymongo import MongoClient
from bson.binary import Binary

# Function to download and store image in MongoDB
def download_and_store_image(url):
    # Download the image from the URL
    response = requests.get(url)
    if response.status_code == 200:
        image_data = response.content

        # Connect to MongoDB
        client = MongoClient('localhost', 27017)
        db = client["Todo"]
        collection = db["CRUD_exercise"]

        # Store the image data in MongoDB
        document = {"image": Binary(image_data)}
        collection.insert_one(document)
        print("Image stored in MongoDB successfully.")
    else:
        print("Failed to download the image.")

# Example usage: provide a URL of the image you want to download and store
image_url = "https://next.ink/wp-content/uploads/2023/12/netflix-nouveau-logo-1536x864.jpg"
download_and_store_image(image_url)


Image stored in MongoDB successfully.


### 14: Try to store a pandas dataframe in mongoDB (array with rownames, array with colnames and matrix with values)

In [6]:
import pandas as pd
from pymongo import MongoClient

# Sample DataFrame
data = {
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
}
df = pd.DataFrame(data)

# Convert DataFrame to dictionary
df_dict = {
    'row_names': df.index.tolist(),
    'col_names': df.columns.tolist(),
    'values': df.values.tolist()
}

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client["Todo"]
collection = db["DataFrame"]

# Insert the dictionary into MongoDB
collection.insert_one(df_dict)
print("DataFrame stored in MongoDB successfully.")


DataFrame stored in MongoDB successfully.


### 15: Insert the movie_review.tsv data into mongodb. Then query it to find the number of review that are positive and negative review. Fetch the docs which have "unexpected" in their review, how many are they ? Think of a clever way to count the number of words in the review using MongoDB (hint: Transform the review text before the insert in MongoDB) and create a density of number of words per review.

In [18]:
import pandas as pd
from pymongo import MongoClient
import re

# Read the TSV file and transform it into a list of dictionaries
df = pd.read_csv('C:/Users/oumis/Downloads/movie_review.tsv', sep='\t')
reviews = df.to_dict(orient='records')

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client["Todo"]
collection = db["movie_reviews"]

# Insert the list of dictionaries into MongoDB
collection.insert_many(reviews, ordered=False)
print("Data inserted into MongoDB successfully.")

# Query the collection to find the number of positive and negative reviews
positive_reviews_count = collection.count_documents({"sentiment": "positive"})
negative_reviews_count = collection.count_documents({"sentiment": "negative"})
print("Number of positive reviews:", positive_reviews_count)
print("Number of negative reviews:", negative_reviews_count)


# Fetch the documents that contain "unexpected" in their review
unexpected_reviews = collection.find({"review": {"$regex": "unexpected"}}, {"_id": 0, "review": 1})
unexpected_reviews_count = 0
for _ in unexpected_reviews:
    unexpected_reviews_count += 1

print("Number of reviews containing 'unexpected':", unexpected_reviews_count)




Data inserted into MongoDB successfully.
Number of positive reviews: 0
Number of negative reviews: 0
Number of reviews containing 'unexpected': 0


### 16: Download a sound sample. Try to store it in MongoDB

In [19]:
import pymongo
from pymongo import MongoClient
import gridfs

# Connect to MongoDB
client = MongoClient('localhost', 27017)
db = client['sound_samples']
fs = gridfs.GridFS(db)

# Function to store sound sample in MongoDB
def store_sound_sample(file_path):
    with open(file_path, 'rb') as file:
        data = file.read()
        fs.put(data, filename=file_path.split('/')[-1])  # Store the file in GridFS
        print("Sound sample stored in MongoDB successfully.")

# Example usage: provide the file path of the sound sample you want to store
file_path = "C:/Users/oumis/Downloads/sound2mongo.wav"
store_sound_sample(file_path)


Sound sample stored in MongoDB successfully.


### 17: Create a collection with 30M observation with a single key : "year" which is a random value between 2000-2020. Get documents with year = 2000. Does using an index helps ?

In [ ]:
import pymongo
import random
import time

# Connect to MongoDB
client = pymongo.MongoClient('localhost', 27017)
db = client['test_db']
collection = db['test_collection']

# Function to generate random "year" values between 2000 and 2020
def generate_random_year():
    return random.randint(2000, 2020)

# Insert 30M documents with random "year" values
start_time = time.time()
for _ in range(30000000):
    document = {"year": generate_random_year()}
    collection.insert_one(document)
end_time = time.time()
print("Time taken to insert 30M documents:", end_time - start_time, "seconds")

# Query documents with "year" equal to 2000 without an index
start_time = time.time()
query_result = collection.find({"year": 2000})
end_time = time.time()
print("Time taken to query documents without an index:", end_time - start_time, "seconds")
print("Number of documents with year = 2000:", query_result.count())

# Create an index on the "year" field
collection.create_index([("year", pymongo.ASCENDING)])

# Query documents with "year" equal to 2000 with an index
start_time = time.time()
query_result = collection.find({"year": 2000})
end_time = time.time()
print("Time taken to query documents with an index:", end_time - start_time, "seconds")
print("Number of documents with year = 2000:", query_result.count())
